In [1]:
import os
import sys
curr_dir=os.getcwd()
p=curr_dir.find('dev')
root=curr_dir[:p]
sys.path.append(root+'lib')
import psi4
import numpy as np
from P4toC4_aux import *
from SO_aux import SymOrbs

In [2]:
#BASIS='STO-3G'
#BASIS='def2-SV'
#BASIS='def2-SVP'
BASIS='CC-PVDZ'

In [3]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", BASIS)
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "RHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)


mol = psi4.geometry("""
0 1
N    0.16600540    -1.33139654    -0.00000000
N   -0.16600540     1.33139654    -0.00000000
H    0.73214469     2.09720536     1.54771041
H    0.73214469     2.09720536    -1.54771041
H   -0.73214469    -2.09720536    -1.54771041
H   -0.73214469    -2.09720536     1.54771041

units Bohr
no_reorient
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-111.17726697487842

In [4]:
basisset=wf.basisset()
#
#  C4_MO[p2c[i]] = P4_MO[i]     P4_MO[c2p[i]] = C4_MO[i]
#
p2c_map, p2c_scale = basis_mapping(basisset, verbose=0)
#c2p_map = invert_mapping(p2c_map)
# Print a new basis set in GENBAS format
#print(basisset.genbas())

In [5]:
mol=wf.molecule()
ptgr=mol.point_group()
print(f'{ptgr.symbol()}: order = {ptgr.order()}')
n_irrep=wf.nirrep()
g=wf.nmopi()
n_mo_pi=np.array(g.to_tuple())
print('MOs per irrep', n_mo_pi)

c2h: order = 4
MOs per irrep [15  9  9 15]


In [6]:
Ls=wf.aotoso()
print(Ls.shape)
# Psi4 MOs in SO basis
C_SO=wf.Ca()
#Cb=np.array(wf.Cb())
C_SO.shape

((48, 15), (48, 9), (48, 9), (48, 15))


((15, 15), (9, 9), (9, 9), (15, 15))

In [7]:
irrep_lst = []

for isym in range(ptgr.order()):
    SOs=SymOrbs(Ls.nph[isym], order=wf.nirrep())
    #SOs.print()
    p4_first_AOs = SOs.first_AOs()
    cfour_first_AOs = p2c_map[SOs.first_AOs()]
    ao_scale = p2c_scale[SOs.first_AOs()]
    so_c2p = np.argsort(cfour_first_AOs)
    nsos=len(so_c2p)
    so_p2c = invert_mapping(so_c2p)
    so_scale=SOs.inv_coef()
    scale = so_scale*ao_scale
    C=psi4_to_c4(C_SO.nph[isym], so_p2c, scale)
    irrep_lst.append(C)
    print(f'\nIrrep {isym}')
    print('AO-order  AO-order   Cfour    argsort    AO     SO')
    print('  Psi4     Cfour    argsort   inverted  scale  scale')
    for i in range(SOs.nsos):
        print(f'{p4_first_AOs[i]:4d}{cfour_first_AOs[i]:9d}', end='')
        print(f'{so_c2p[i]:11d}{so_p2c[i]:10d}', end='')
        print(f'{ao_scale[i]:11.3f}{so_scale[i]:7.3f}')
    
C_SOr = psi4.core.Matrix.from_array(irrep_lst)
C_SOr.shape


Irrep 0
AO-order  AO-order   Cfour    argsort    AO     SO
  Psi4     Cfour    argsort   inverted  scale  scale
   0        0          0         0      1.000  1.414
   1        1          1         1      1.000  1.414
   2        2          2         2      1.000  1.414
   4        3          3         3      1.000  1.414
   5        5          5         5      1.000  1.414
   7        4          4         4      1.000  1.414
   8        6          6         6      1.000  1.414
   9        9          7         7      3.464  1.414
  12       12          9         9      2.000  1.414
  13       10          8         8      1.000  1.414
  28       28         10        10      1.000  2.000
  29       29         11        11      1.000  2.000
  30       32         13        14      1.000  2.000
  31       30         14        12      1.000  2.000
  32       31         12        13      1.000  2.000

Irrep 1
AO-order  AO-order   Cfour    argsort    AO     SO
  Psi4     Cfour    argsort   in

((15, 15), (9, 9), (9, 9), (15, 15))

In [8]:
p2c_irrep_map=np.array([0,2,3,1])
c2p_irrep_map=invert_mapping(p2c_irrep_map)
print(p2c_irrep_map)
C4_cs = read_oldmos('OLDMOS.'+BASIS, n_mo_pi[p2c_irrep_map])

[0 2 3 1]
reading orbitals from OLDMOS.CC-PVDZ


In [9]:
cfour_sym=3
psi4_sym=p2c_irrep_map[cfour_sym]
Corg=C_SO.nph[psi4_sym]
Creo=C_SOr.nph[psi4_sym]
Cc4=C4_cs[cfour_sym]
naos=n_mo_pi[psi4_sym]
mo=1
print('          Psi4    reordered    Cfour')
for k in range(naos):
    print(f'{k:3d}  {Corg[k,mo]:10.6f} {Creo[k,mo]:10.6f} {Cc4[k,mo]:10.6f}')
print(np.max(Creo[:,mo]-Cc4[:,mo]))

          Psi4    reordered    Cfour
  0    0.309062   0.218540   0.218540
  1    0.999425   0.706700   0.706700
  2   -0.023172  -0.016385  -0.016385
  3   -0.013799  -0.009758  -0.009758
  4    0.048167   0.024083   0.024083
  5    2.316756   1.158378   1.158378
  6   -0.031997  -0.006103  -0.006103
  7   -0.012205  -0.012998  -0.012998
  8   -0.025995  -0.015999  -0.015999
2.2407992128492538e-09


In [10]:
#
#  comparison Psi4-MOs and Cfour-MOs in their SO representation
#
for cfour_irrep in range(wf.nirrep()):
    psi4_irrep=p2c_irrep_map[cfour_irrep]
    print(cfour_irrep, psi4_irrep, C4_cs[cfour_irrep].shape, C_SOr.nph[psi4_irrep].shape )
    print(np.max(abs(C_SOr.nph[psi4_irrep])-abs(C4_cs[cfour_irrep])))

0 0 (15, 15) (15, 15)
1.424822251849811e-08
1 2 (9, 9) (9, 9)
5.671611222335393e-09
2 3 (15, 15) (15, 15)
8.914353505362627e-09
3 1 (9, 9) (9, 9)
7.047298744566888e-09
